In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate some sample electron velocity data
velocities = np.random.normal(loc=0, scale=1, size=10000)

# Calculate the empirical CDF
x = np.sort(velocities)
y = np.arange(1, len(velocities)+1) / len(velocities)

# Plot the CDF
plt.plot(x, y)
plt.xlabel('Electron Velocity (m/s)')
plt.ylabel('Cumulative Probability')
plt.title('Electron Velocity Distribution')
plt.show()

In [ ]:
plt.plot(x, y)
plt.xlabel('Electron Velocity (m/s)')
plt.ylabel('Cumulative Probability')
plt.title('Electron Velocity Distribution')
plt.show()

In [ ]:
import spacepy.pycdf as pycdf
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from operator import length_hint
from cdasws import CdasWs
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
from plasmapy.particles import *
from plasmapy.formulary import *
import astropy as astropy
import numpy as np
from datetime import date

from astropy import units as u
from scipy.special import gamma
import scipy.integrate as integrate

from plasmapy.formulary import parameters

# # Read data from CDF file
# cdf_file = pycdf.CDF('wi_elpd_3dp_20230328_v02.cdf')
# energy = cdf_file['ENERGY'][...]
# pitch_angle = cdf_file['PANGLE'][...]
# flux = cdf_file['FLUX'][...]
# cdf_file.close()

## Getting data from WI_ELDP_3DP
params = ['ENERGY','PANGLE','FLUX']

# get_data([__], [__], [_start-time_], [_end-time_])
# start/end-time format: [_Year_]-[_month_]-[_day_]T[_hour_]:[_minute_]:[_second_].[_millisecond_]Z
stat, data = CdasWs().get_data('WI_ELDP_3DP', params, '1999-07-12T00:00:00.000Z', '2000-07-12T00:00:00.000Z')

## Get number of datapoints available 
datalength = len(data['PANGLE'])

# electron_Energy contains the energy of incoming electrons
electron_Energy = list(data['ENERGY'])

# electron_PitchAngle contains the pitch angle of incoming electrons
electron_PitchAngle = list(data['PANGLE'])

# electron_Flux contains the flux of incoming electrons
electron_Flux = list(data['FLUX'])

theta = np.arccos(np.sqrt(1 - (B/B0)**2) * np.cos(electron_PitchAngle))

print("***datalength****************")
print(datalength)
print("***ENERGY****************")
print(electron_Energy)
print("***PANGLE****************")
print(electron_PitchAngle)
print("***FLUX****************")
print(electron_Flux)

# # Define the grid
# energy_grid = np.linspace(0, 30, 120)
# pitch_angle_grid = np.linspace(0, 180, 15)
# energy_grid, pitch_angle_grid = np.meshgrid(energy_grid, pitch_angle_grid)

# # Interpolate the flux values onto the grid
# flux_interp = griddata((pitch_angle.flatten(), energy.flatten()), flux.flatten(),
#                        (energy_grid, pitch_angle_grid), method='linear')

# # Plot the 2D histogram
# fig, ax = plt.subplots()
# im = ax.imshow(flux_interp.T, extent=[0, 180, 0, 30], origin='lower',
#                aspect='auto', cmap='inferno')
# ax.set_xlabel('Pitch Angle (deg)')
# ax.set_ylabel('Energy (keV)')
# cbar = fig.colorbar(im)
# cbar.set_label('Flux')
# plt.show()

In [ ]:
from cdasws import CdasWs
from datetime import datetime
import tzlocal

cdas = CdasWs()

# Import data from the MFI instrument
varsMFI = ['BF1','BF1LOG','BRMSF1','BGSM','BRMSGSM','BGSE','BGSEa','BRMSGSE','DIST','PGSM','PGSE','B3F1','B3F1LOG','B3RMSF1','B3GSM','B3RMSGSM','B3GSE','B3GSEa','B3RMSGSE','B1F1','B1F1LOG','B1RMSF1','B1GSM','B1RMSGSM','B1GSE','B1GSEa','B1RMSGSE','DIST1','P1GSM','P1GSE','DISTV','PGSMV','PGSEV','DIST1V','P1GSMV','P1GSEV']

# Import data from the 3DP instrument
vars3DP = ['FLUX','FLUX_byE_atA','FLUX_byE_atA_stackPA0','FLUX_byE_atA_stackPA1','FLUX_byE_atA_stackPA2','FLUX_byE_atA_stackPA3','FLUX_byE_atA_stackPA4','FLUX_byE_atA_stackPA5','FLUX_byE_atA_stackPA6','FLUX_byE_atA_stackPA7','FLUX_byA_atE','FLUX_byA_atE_stackE0','FLUX_byA_atE_stackE2','FLUX_byA_atE_stackE4','FLUX_byA_atE_stackE6','FLUX_byA_atE_stackE8','FLUX_byA_atE_stackE10','FLUX_byA_atE_stackE12','FLUX_byA_atE_stackE14','PANGLE','ENERGY','VSW','MAGF','TIME']

# Set time interval to pull data from
time = ['1995-06-19T00:00:00.000Z', '1995-06-20T00:00:00.000Z']
status3DP, data3DP = cdas.get_data('WI_ELPD_3DP', vars3DP, time[0], time[1])
statusMFI, dataMFI = cdas.get_data('WI_H0_MFI', varsMFI, time[0], time[1])


# Data from MFI instrument
magB_3sec = dataMFI['B3F1'] # magB_3sec = Magnetic field magnitude taken at 3 second intervals
vectorB_GSEcart_3sec = dataMFI['B3GSE'] # vectorB_GSEcart_3sec = Magnetic field vector in GSE cartesian coordinates taken at 3 second intervals
vectorB_GSEang_3sec = dataMFI['B3GSEa'] # = vectorB_GSEang_3sec = Magnetic field vector in GSE angular coordinates taken at 3 second intervals
vectorPostition_GSE_1min = dataMFI['PGSEV'] # vectorPostition_GSE_1min = Time axis label: Position vector in GSE coordinates (Define Re = 6378km) taken at 1 minute intervals

# Data from 3DP instrument
timeUNIX = data3DP['TIME'] # timeUNIX = time the data was taken [UNIX time]
energy = data3DP['ENERGY'] # energy = energy of incoming electrons in [eV]
flux = data3DP['FLUX'] # flux = flux of the electrons

print(datetime.fromtimestamp(data3DP['TIME'][1]))

# variable to syncronize the output data (MAX = 70; MIN = 0)
sync = 1

# print(len(timeUNIX))
# print('***************')
# print(len(flux))
# print('***************')
# print(len(energy))
# print('***************')
# print('***************')
print(len(magB_3sec))
# print('***************')
# print(len(vectorB_GSEcart_3sec))
# print('***************')
# print(len(vectorB_GSEang_3sec))
# print('***************')
# print(len(vectorPostition_GSE_1min))
# print('***************')
# print('***************')
print(magB_3sec)


In [6]:
from cdasws import CdasWs
from datetime import datetime
import tzlocal
import numpy as np

cdas = CdasWs()

# Import data from the MFI instrument
varsMFI = ['BF1','BF1LOG','BRMSF1','BGSM','BRMSGSM','BGSE','BGSEa','BRMSGSE','DIST','PGSM','PGSE','B3F1','B3F1LOG','B3RMSF1','B3GSM','B3RMSGSM','B3GSE','B3GSEa','B3RMSGSE','B1F1','B1F1LOG','B1RMSF1','B1GSM','B1RMSGSM','B1GSE','B1GSEa','B1RMSGSE','DIST1','P1GSM','P1GSE','DISTV','PGSMV','PGSEV','DIST1V','P1GSMV','P1GSEV']

# Import data from the 3DP instrument
vars3DP = ['FLUX','FLUX_byE_atA','FLUX_byE_atA_stackPA0','FLUX_byE_atA_stackPA1','FLUX_byE_atA_stackPA2','FLUX_byE_atA_stackPA3','FLUX_byE_atA_stackPA4','FLUX_byE_atA_stackPA5','FLUX_byE_atA_stackPA6','FLUX_byE_atA_stackPA7','FLUX_byA_atE','FLUX_byA_atE_stackE0','FLUX_byA_atE_stackE2','FLUX_byA_atE_stackE4','FLUX_byA_atE_stackE6','FLUX_byA_atE_stackE8','FLUX_byA_atE_stackE10','FLUX_byA_atE_stackE12','FLUX_byA_atE_stackE14','PANGLE','ENERGY','VSW','MAGF','TIME']

# Set time interval to pull data from
time = ['1995-06-19T00:00:00.000Z', '1995-06-20T00:00:00.000Z']
status3DP, data3DP = cdas.get_data('WI_ELPD_3DP', vars3DP, time[0], time[1])
statusMFI, dataMFI = cdas.get_data('WI_H0_MFI', varsMFI, time[0], time[1])


# Data from MFI instrument
MFI_time = [8.03520012e+08+i for i in range(28800)]
magB_3sec = dataMFI['B3F1'] # magB_3sec = Magnetic field magnitude taken at 3 second intervals
vectorB_GSEcart_3sec = dataMFI['B3GSE'] # vectorB_GSEcart_3sec = Magnetic field vector in GSE cartesian coordinates taken at 3 second intervals
vectorB_GSEang_3sec = dataMFI['B3GSEa'] # = vectorB_GSEang_3sec = Magnetic field vector in GSE angular coordinates taken at 3 second intervals
vectorPostition_GSE_1min = dataMFI['PGSEV'] # vectorPostition_GSE_1min = Time axis label: Position vector in GSE coordinates (Define Re = 6378km) taken at 1 minute intervals

# Data from 3DP instrument
timeUNIX = data3DP['TIME'] # timeUNIX = time the data was taken [UNIX time]
energy_eV = data3DP['ENERGY'] # energy = energy of incoming electrons in [eV]
flux = data3DP['FLUX'] # flux = flux of the electrons
mass_e_kg = 9.109e-31 # Mass of an electron in kg






# energy_J = energy_eV * 1.60218e-19
# magB_T = magB_3sec * 1e-9
# velocity =  (2*energy_J/mass_e_kg) ** 0.5
# gyrofreq = abs(1.60217662e-19 * magB_T / (9.10938356e-31 * 2 * np.pi))



# # Synchronize time stamps
# timevectorB_GSEcart_3sec = MFI_time # Time axis label: Time
# timevectorvelocity_1min = data3DP['TIME'] # Time axis label: Time

# # Find common time range
# common_time = np.intersect1d(timevectorB_GSEcart_3sec, timevectorvelocity_1min)

# # Filter data by common time range
# indexvectorB_GSEcart_3sec = np.isin(timevectorB_GSEcart_3sec, common_time)
# indexvectorvelocity_1min = np.isin(timevectorvelocity_1min, common_time)
# vectorspeed = dataMFI['PGSE'][1:,:] - dataMFI['PGSE'][:-1,:]

# # Resize vectors to match shape
# if vectorspeed.shape[0] > vectorB_GSEcart_3sec.shape[0]:
#     vectorspeed = vectorspeed[::int(vectorspeed.shape[0]/vectorB_GSEcart_3sec.shape[0])]
# elif vectorspeed.shape[0] < vectorB_GSEcart_3sec.shape[0]:
#     vectorB_GSEcart_3sec = vectorB_GSEcart_3sec[::int(vectorB_GSEcart_3sec.shape[0]/vectorspeed.shape[0])]

# # Calculate angle between velocity and magnetic field
# cosalpha = np.abs(np.dot(vectorspeed, vectorB_GSEcart_3sec.T)) / (np.linalg.norm(vectorspeed, axis=1) * np.linalg.norm(vectorB_GSEcart_3sec, axis=1))
# alpha = np.arccos(cosalpha)

# cosalpha = np.abs(np.dot(vectorspeed, vectorB_GSEcart_3sec.T)) / (np.linalg.norm(vectorspeed, axis=1) * np.linalg.norm(vectorB_GSEcart_3sec, axis=1))
# alpha = np.arccos(cosalpha)

# vperp = velocity * np.sin(alpha)
# vparallel = velocity * np.cos(alpha)





# # variable to syncronize the output data (MAX = 70; MIN = 0)
# sync = 1

print(energy_eV[0])
print(energy_eV[1])
print(energy_eV[70])
print('***************')


[1170.5605    843.62756   608.03015   438.38284   316.10492   228.0873
  164.70824   119.16503    86.31407    62.588425   45.58229    33.38769
   24.59424    18.206553   13.643932]
[1170.5605    843.62756   608.03015   438.38284   316.10492   228.0873
  164.70824   119.16503    86.31407    62.588425   45.58229    33.38769
   24.59424    18.206553   13.643932]
[1170.5605    843.6276    608.03015   438.38284   316.10492   228.0873
  164.70824   119.16503    86.314064   62.588425   45.58229    33.38769
   24.59424    18.206553   13.643932]
***************
